# Second LLM Pass (or labeling large text portions) research
As initial attempts to create e2e pipeline resulted in low quality results we want to know other options on design. Here I will gather info about possibilities to use ML tools to categorize section of text (possibly xml/csv) as coupon.

## Similar existing problems
### Sentence classification
There are many works on classifying single sequences and datasets focused on this task (ex [trec](https://huggingface.co/datasets/CogComp/trec)).
### Larger text classification
Closer to our problem, we have reviews classification task. Reviews can be quite [long](https://medium.com/codex/fine-tune-bert-for-text-classification-cef7a1d6cdf1), but we still miss two aspects of our problem: we do not know where possible coupons have beginning and end ending, which stops us from applying this method directly.
### Text Retrieval and Question answering
There are tasks of question answering and text retrieval, more similar to our problem. Given portion of text and possibly question, model selects begin and end of sequence containing answer. This addresses our lack of knowledge about exact coupon placement. Despite that, we still have to deal with the fact that we might have more than one coupon in one provided xml portion. I will discuss this problem further in following section. \\
Note: text retrieval seems to be quite niche, there is even no model task on HF for that, only for datasets.
## Known challenges
### Random coupon placement
Coupons might be located in groups, there will also be large sparse portions of text without any.This presents a challenge both to the QA and standard classification approach. Possible solutions:
* combining with heuristic used in first PoC (analyzing count of labels from first BERT pass and decide whether xml node is possibly a coupon, and it is worth inputting to classification model.
* simply chunking text and make two passes to avoid having coupon on chunk border
* multi-span question answering: there are [attempts](https://aclanthology.org/2020.emnlp-main.248.pdf) to modify qa task to produce multiple outputs - provided example is actually close to what we were discussed on meetings (casting QA task to NER-like token labeling)
### Providing input in reasonable format
Possibly feeding LLM with raw CSV is not the best idea. It contains a lot of boilerplate data and probably does not represent xml structure well.

## DOZRO
* research if (and if yes, how) LLMs can handle csv/xml inputs. do we need some token mapping of xml tags?
* add some suggestions (is option good/bad, why)
* more options